# Challenge Exercise, now with hyperparameter tuning via Cloud ML Engine 

Create a neural network that is capable of finding the volume of a cylinder given the radius of its base (r) and its height (h). Assume that the radius and height of the cylinder are both in the range 0.5 to 2.0. Unlike in the challenge exercise for b_estimator.ipynb, assume that your measurements of r, h and V are all rounded off to the nearest 0.1. Simulate the necessary training dataset. This time, you will need a lot more data to get a good predictor.
<p>
Now modify the "noise" so that instead of just rounding off the value, there is up to a 10% error (uniformly distributed) in the measurement followed by rounding off.

## Set up environment

In [17]:
PROJECT = 'cylinders'
BUCKET = 'cylinders'
REGION = 'europe-west1'

In [18]:
# for bash
import os
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.8'  # Tensorflow version

In [19]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


## Generate cylinders and upload to bucket

In [46]:
%%bash
# Generate cylinders and upload to bucket
python generate_cylinders.py --filename "cylinders_train.csv" --size 8000
python generate_cylinders.py --filename "cylinders_eval.csv" --size 1000
python generate_cylinders.py --filename "cylinders_test.csv" --size 1000
# gsutil mv "cylinders_*.csv" "gs://$BUCKET/input/"

saved 8000 cylinders to cylinders_train.csv
saved 1000 cylinders to cylinders_eval.csv
saved 1000 cylinders to cylinders_test.csv


## Create command-line program

In order to submit to Cloud ML Engine, we need to create a distributed training program. Let's convert our housing example to fit that paradigm, using the Estimators API.

In [21]:
%%bash
rm -rf cylinder_prediction_module
mkdir cylinder_prediction_module
mkdir cylinder_prediction_module/trainer
touch cylinder_prediction_module/trainer/__init__.py

### task.py

In [22]:
%%writefile cylinder_prediction_module/trainer/task.py
import argparse
import os
import json
import shutil

from . import model
    
if __name__ == '__main__' and 'get_ipython' not in dir():
    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--learning_rate',
        type = float, 
        default = 0.01
    )
    parser.add_argument(
        '--batch_size',
        type = int, 
        default = 30
    )
    parser.add_argument(
        '--output_dir',
        help = 'GCS location to write checkpoints and export models.',
        required = True
    )
    parser.add_argument(
        '--job-dir',
        help = 'this model ignores this field, but it is required by gcloud',
        default = 'junk'
    )
    args = parser.parse_args()
    arguments = args.__dict__

    # Unused args provided by service
    arguments.pop('job_dir', None)
    arguments.pop('job-dir', None)

    # Append trial_id to path if we are doing hptuning
    # This code can be removed if you are not using hyperparameter tuning
    arguments['output_dir'] = os.path.join(
        arguments['output_dir'],
        json.loads(
            os.environ.get('TF_CONFIG', '{}')
        ).get('task', {}).get('trial', '')
    )

    # Run the training
    shutil.rmtree(arguments['output_dir'], ignore_errors=True) # start fresh each time

    # Pass the command line arguments to our model's train_and_evaluate function
    model.train_and_evaluate(arguments)

Writing cylinder_prediction_module/trainer/task.py


### model.py

In [51]:
%%writefile cylinder_prediction_module/trainer/model.py
import google.datalab.storage as storage
from io import BytesIO
import numpy as np
import pandas as pd
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

# Read cylinder CSVs from bucket
traindf = pd.read_csv('cylinders_train.csv')
evaldf = pd.read_csv('cylinders_eval.csv')

# Train and eval input functions
def train_input_fn(df, batch_size):
    return tf.estimator.inputs.pandas_input_fn(
        x = traindf,
        y = traindf['volume'],
        num_epochs = None,
        batch_size = batch_size,
        shuffle = True)

def eval_input_fn(df, batch_size):
    return tf.estimator.inputs.pandas_input_fn(
        x = evaldf,
        y = evaldf['volume'],
        num_epochs = 1,
        batch_size = batch_size,
        shuffle = False)

# Define feature columns
features = [
    tf.feature_column.numeric_column(key='radius', dtype=tf.float64),
    tf.feature_column.numeric_column(key='height', dtype=tf.float64)]

def train_and_evaluate(args):
    # Compute appropriate number of steps.
    num_steps = (len(traindf) / args['batch_size']) / args['learning_rate']
    # Thus, if learning_rate = 0.01, hundred epochs

    # Create custom optimizer
    myopt = tf.train.FtrlOptimizer(learning_rate=args['learning_rate'])

    # Create rest of the estimator as usual
    estimator = tf.estimator.LinearRegressor(
        model_dir = args['output_dir'], 
        feature_columns = features, 
        optimizer = myopt)
    
    #Add rmse evaluation metric
    def rmse(labels, predictions):
        pred_values = tf.cast(predictions['predictions'], tf.float64)
        return {'rmse': tf.metrics.root_mean_squared_error(labels, pred_values)}

    estimator = tf.contrib.estimator.add_metrics(estimator, rmse)

    train_spec = tf.estimator.TrainSpec(
        input_fn = train_input_fn(df = traindf, batch_size = args['batch_size']),
        max_steps = num_steps)

    eval_spec = tf.estimator.EvalSpec(
        input_fn = eval_input_fn(df = evaldf, batch_size = len(evaldf)),
        steps = None)

    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

Overwriting cylinder_prediction_module/trainer/model.py


## Monitoring with TensorBoard
Use "refresh" in Tensorboard during training to see progress.

In [54]:
from google.datalab.ml import TensorBoard
OUTDIR = './cylinders_trained'
TensorBoard().start(OUTDIR)

TensorBoard was started successfully with pid 4985. Click here to access it.

4985

### Shut TensorBoard down

In [57]:
pids_df = TensorBoard.list()
if not pids_df.empty:
    for pid in pids_df['pid']:
        TensorBoard().stop(pid)
        print('Stopped TensorBoard with pid {}'.format(pid))

Stopped TensorBoard with pid 4985


### Train the model locally to see if everything works

In [52]:
%%bash
rm -rf cylinders_trained
export PYTHONPATH=${PYTHONPATH}:${PWD}/cylinder_prediction_module
gcloud ml-engine local train \
    --module-name=trainer.task \
    --job-dir=cylinders_trained \
    --package-path=$(pwd)/trainer \
    -- \
    --batch_size=30 \
    --learning_rate=0.02 \
    --output_dir=cylinders_trained

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:TF_CONFIG environment variable: {'cluster': {}, 'job': {'job_name': 'trainer.task', 'args': ['--batch_size=30', '--learning_rate=0.02', '--output_dir=cylinders_trained', '--job-dir', 'cylinders_trained']}, 'task': {}, 'environment': 'cloud'}
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_log_step_count_steps': 100, '_save_summary_steps': 100, '_service': None, '_model_dir': 'cylinders_trained/', '_session_config': None, '_train_distribute': None, '_keep_checkpoint_every_n_hours': 10000, '_global_id_in_cluster': 0, '_task_type': 'worker', '_num_worker_replicas': 1, '_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5, '_task_id': 0, 

## Create hyperparam.yaml

In [55]:
%%writefile hyperparam.yaml
trainingInput:
    hyperparameters:
        goal: MINIMIZE
        maxTrials: 16
        maxParallelTrials: 2
        hyperparameterMetricTag: rmse
        params:
        - parameterName: batch_size
          type: INTEGER
          minValue: 8
          maxValue: 64
          scaleType: UNIT_LINEAR_SCALE
        - parameterName: learning_rate
          type: DOUBLE
          minValue: 0.01
          maxValue: 0.1
          scaleType: UNIT_LOG_SCALE

Writing hyperparam.yaml


## Submit the hyperparameter tuning job to Cloud ML Engine

In [59]:
%%bash
OUTDIR=gs://${BUCKET}/cylinders_trained   # CHANGE bucket name appropriately
gsutil rm -rf $OUTDIR
export PYTHONPATH=${PYTHONPATH}:${PWD}/cylinder_prediction_module
gcloud ml-engine jobs submit training cylinders_$(date -u +%y%m%d_%H%M%S) \
    --config=hyperparam.yaml \
    --module-name=trainer.task \
    --package-path=$(pwd)/cylinder_prediction_module/trainer \
    --job-dir=$OUTDIR \
    --runtime-version=$TFVERSION \
    --\
    --output_dir=$OUTDIR \

jobId: cylinders_190314_133749
state: QUEUED


Removing gs://cylinders/cylinders_trained/packages/b1b5b353b930eabdda02a904c7db491433eda70f05f4286af4c78b738f72dc8d/trainer-0.0.0.tar.gz#1552570494328430...
/ [1 objects]                                                                   
Operation completed over 1 objects.                                              
Job [cylinders_190314_133749] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe cylinders_190314_133749

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs cylinders_190314_133749


### Monitor the job

In [60]:
!gcloud ml-engine jobs describe cylinders_190314_133749  # Change jobId to what the previous cell output

createTime: '2019-03-14T13:37:52Z'
etag: X6m_anszLjA=
jobId: cylinders_190314_133749
startTime: '2019-03-14T13:37:55Z'
state: RUNNING
trainingInput:
  args:
  - --output_dir=gs://cylinders/cylinders_trained
  hyperparameters:
    goal: MINIMIZE
    hyperparameterMetricTag: rmse
    maxParallelTrials: 2
    maxTrials: 16
    params:
    - maxValue: 64.0
      minValue: 8.0
      parameterName: batch_size
      scaleType: UNIT_LINEAR_SCALE
      type: INTEGER
    - maxValue: 0.1
      minValue: 0.01
      parameterName: learning_rate
      scaleType: UNIT_LOG_SCALE
      type: DOUBLE
  jobDir: gs://cylinders/cylinders_trained
  packageUris:
  - gs://cylinders/cylinders_trained/packages/77e32208113769baa0952b8a4950a8e090ff2b5d850704214e3388beb06e9b63/trainer-0.0.0.tar.gz
  pythonModule: trainer.task
  region: europe-west1
  runtimeVersion: '1.8'
trainingOutput:
  isHyperparameterTuningJob: true

View job in the Cloud Console at:
https://console.cloud.google.com/ml/jobs/cylinders_190314_13